In [186]:
import pandas as pd


In [187]:
#목표:
#병원, 약국 부족량 공식을 만들고
#이 값이 클 수록 병원, 약국이 부족한 지역으로 본다.
#이 값을 y로 잡고 다른 변수들을 x로 잡아서 회귀분석을 한다.

In [209]:
# 데이터 가져오기
hospital_df = pd.read_csv('data/hospitals.csv')
pharmacy_df = pd.read_csv('data/pharmacies.csv')
pet_df = pd.read_csv('data/pet_data.csv')
hospital_df.head(4)

,구별,동별,사업장명,소재지도로명주소,위도,경도
0,분당구,구미동,분당필동물병원,경기도 성남시 분당구 금곡로15번길 1 (구미동),37.343838,127.110754
1,분당구,구미동,현대 동물병원,"경기도 성남시 분당구 성남대로 30, 111호 (구미동, 동아그린프라자)",37.338343,127.109435
2,분당구,구미동,24시분당리더스동물의료원,"경기도 성남시 분당구 성남대로 45, 계영빌딩 202, 203호 (구미동)",37.340012,127.108392
3,분당구,구미동,한국동물선진의료연구소(KASMINE) 동물심혈관신장센터,"경기도 성남시 분당구 미금로 43, 정운빌딩 1층 (구미동)",37.337094,127.115965


In [210]:
# 병원 및 약국 개수 계산
hospital_counts = hospital_df.groupby(["구별", "동별"]).size().reset_index(name="병원 개수")
pharmacy_counts = pharmacy_df.groupby(["구별", "동별"]).size().reset_index(name="약국 개수")

In [211]:
# 병원, 약국 개수를 반려동물 데이터와 병합
merged_df = pet_df.merge(hospital_counts, on=["구별", "동별"], how="left") \
                    .merge(pharmacy_counts, on=["구별", "동별"], how="left")

In [212]:
merged_df.head(4)

,구별,동별,동물소유자수,반려동물수,병원 개수,약국 개수
0,분당구,구미동,2523,3241,7.0,12.0
1,분당구,궁내동,160,217,NaN,1.0
2,분당구,금곡동,1629,2091,4.0,11.0
3,분당구,대장동,219,303,2.0,2.0


In [213]:
#결측치를 0으로 채워넣기
merged_df = merged_df.fillna(0)

In [215]:
merged_df.isna().sum()

구별        0
동별        0
동물소유자수    0
반려동물수     0
병원 개수     0
약국 개수     0
dtype: int64

In [200]:
#병원 부족량 공식: (반려동물수 + 인구밀도) / (병원 개수 + 1)
count_df["병원 부족량"] = (count_df["반려동물수"] + count_df["동물소유자수"]) / (count_df["병원 개수"] + 1)
count_df["병원 부족량"] = count_df["병원 부족량"].apply(lambda x: max(0, x - 1))  # 병원이 많을수록 부족량 감소

In [150]:
count_df.isna().sum()

구별        0
동별        0
병원 개수     0
동물소유자수    0
반려동물수     0
병원 부족량    0
dtype: int64

In [151]:
count_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구별      29 non-null     object 
 1   동별      29 non-null     object 
 2   병원 개수   29 non-null     int64  
 3   동물소유자수  29 non-null     int64  
 4   반려동물수   29 non-null     int64  
 5   병원 부족량  29 non-null     float64
dtypes: float64(1), int64(3), object(2)
memory usage: 1.5+ KB


In [202]:
# 구별, 동별 원핫 인코딩
from sklearn.preprocessing import OneHotEncoder, StandardScaler

one_hot_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
encoded_features = one_hot_encoder.fit_transform(count_df[["구별", "동별"]])

In [203]:
encoded_df = pd.DataFrame(encoded_features, columns=one_hot_encoder.get_feature_names_out(["구별", "동별"]))
count_df = pd.concat([count_df, encoded_df], axis=1)

In [204]:
#구별, 동별 컬럼 삭제
count_df.drop(["구별", "동별"], axis=1, inplace=True)

In [205]:
from sklearn.model_selection import train_test_split
import xgboost as xgb